# BERT - Out of the Box

In this notebook, we will test the performance of an out-of-the-box BERT model on CommonsenseQA. I follow the tutorial here: https://github.com/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb

I've implemented the Hugginface transformers library. 

I referred to the Commonsense QA repo and code to understand how the authors of this work establiahsed their baseline using BERT. This is the link to their code: https://github.com/jonathanherzig/commonsenseqa/blob/master/bert/run_commonsense_qa.py

From this repo (README): https://github.com/jonathanherzig/commonsenseqa

Their work is far more advanced and complicated than maybe what I want to do at this time. But I refer to their work to understand the set up.

In [1]:
import numpy as np
import pandas as pd
import warnings
import json
from pandas.io.json import json_normalize
warnings.filterwarnings('ignore')
import time 
import pickle
from tensorflow.keras.callbacks import TensorBoard

import logging
import os
import argparse
import random
from tqdm import tqdm, trange
import csv

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

#  PyTorch reimplementation of Google's TensorFlow repository for the BERT model 
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForMultipleChoice
from pytorch_pretrained_bert.optimization import BertAdam
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE

runtype="tiny"

print("Run type:", runtype)
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# print("Num CPUs Available: ",
#       len(tf.config.experimental.list_physical_devices('CPU')))


Run type: tiny


In [2]:
try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter


logger = logging.getLogger(__name__)

## Import dataset

It's in the dataset folder.

In [3]:
def load_data(file):
    lines = []
    with open(file, 'rb') as json_file:
        for json_line in json_file:
            lines.append(json.loads(json_line))
        data = json_normalize(lines)
        data.columns = data.columns.map(lambda x: x.split(".")[-1])
    return data
# os.chdir('w266-commonsenseqa/BERT_oob)
train = load_data('../dataset/train_rand_split.jsonl')
dev = load_data('../dataset/dev_rand_split.jsonl')
train.head()

,answerKey,id,question_concept,choices,stem
0,A,075e483d21c29a511267ef62bedc0461,punishing,"[{'label': 'A', 'text': 'ignore'}, {'label': '...",The sanctions against the school were a punish...
1,B,61fe6e879ff18686d7552425a36344c8,people,"[{'label': 'A', 'text': 'race track'}, {'label...",Sammy wanted to go to where the people were. ...
2,A,4c1cb0e95b99f72d55c068ba0255c54d,choker,"[{'label': 'A', 'text': 'jewelry store'}, {'la...",To locate a choker not located in a jewelry bo...
3,D,02e821a3e53cb320790950aab4489e85,highway,"[{'label': 'A', 'text': 'united states'}, {'la...",Google Maps and other highway and street GPS s...
4,C,23505889b94e880c3e89cff4ba119860,fox,"[{'label': 'A', 'text': 'pretty flowers.'}, {'...","The fox walked from the city into the forest, ..."


In [4]:
train.iloc[0].stem

'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?'

In [5]:
for n in range(1,10):
    print(n, train.iloc[n].stem)

1 Sammy wanted to go to where the people were.  Where might he go?
2 To locate a choker not located in a jewelry box or boutique where would you go?
3 Google Maps and other highway and street GPS services have replaced what?
4 The fox walked from the city into the forest, what was it looking for?
5 What home entertainment equipment requires cable?
6 The only baggage the woman checked was a drawstring bag, where was she heading with it?
7 The forgotten leftovers had gotten quite old, he found it covered in mold in the back of his what?
8 What do people use to absorb extra ink from a fountain pen?
9 Where is a business restaurant likely to be located?


## Steps

1. Import training examples
2. Process it
    - Format input into something BERT can work with, including `[CLS]` and `[SEP]`
    - We were thinking which label is correct: 
    - Tokenize 
    - Create an output layer using softmax. 
3. Train it
    - Specify how many layers of BERT to fine tune
    

# Bert for multiple choice

Source: https://github.com/google-research/bert/issues/38

> Let's assume your batch size is 8 and your sequence length is 128. Each SWAG example has 4 entries, the correct one and 3 incorrect ones.
> 
> Instead of your input_fn returning an input_ids of size [128], it should return one of size [4, 128]. Same for mask and sequence ids. So for each example, you will generate the sequences predicate ending0, predicate ending1, predicate ending2, predicate ending3. Also return a label scalar which is in an integer in the range [0, 3] to indicate what the gold ending is.
> 
> After batching, your model_fn will get an input of shape [8, 4, 128]. Reshape these to [32, 128] before passing them into BertModel. I.e., BERT will consider all of these independently.
> 
> Compute the logits as in run_classifier.py, but your "classifier layer" will just be a vector of size [768] (or whatever your hidden size is).
> 
> Now you have a set of logits of size [32]. Re-shape these back into [8, 4] and then compute tf.nn.log_softmax() over the 4 endings for each example. Now you have log probabilities of shape [8, 4] over the 4 endings and a label tensor of shape [8], so compute the loss exactly as you would for a classification problem.


Source: https://github.com/huggingface/transformers/pull/96/files

> 6. `BertForMultipleChoice`
>
> `BertForMultipleChoice` is a fine-tuning model that includes `BertModel` and a linear layer on top of the `BertModel`.
> 
> The linear layer outputs a single value for each choice of a multiple choice problem, then all the outputs corresponding to an instance are passed through a softmax to get the model choice.
> 
> This implementation is largely inspired by the work of OpenAI in [Improving Language Understanding by Generative Pre-Training](https://blog.openai.com/language-unsupervised/) and the answer of Jacob Devlin in the following [issue](https://github.com/google-research/bert/issues/38).
> 
> An example on how to use this class is given in the [`run_swag.py`](./examples/run_swag.py) script which can be used to fine-tune a multiple choice classifier using BERT, for example for the Swag task.

I'm going to be following the code in this repo very closely: https://github.com/rodgzilla/pytorch-pretrained-BERT/tree/dcb50eaa4b80d3ab75d373c36780c80fb47cfd97

For each question, there are five answer choices. Only one of them is correct.

For BERT, the first thought was to have all five answers attached to each question, and the model would choose one of the five responses. This is how it's originally done in the CommonsenseQA paper.

```
[CLS] Question text here [SEP] Ans choice A [SEP] Ans choice B [SEP] Ans choice C [SEP] Ans choice D [SEP] Ans choice E [SEP]
```

It seems complicated, however, and requires a significant lift. So for now, let me try creating five question-answer pairs for each question. Like this:

```
[CLS] Question text here [SEP] Ans choice A [SEP]
[CLS] Question text here [SEP] Ans choice B [SEP]
[CLS] Question text here [SEP] Ans choice C [SEP]
[CLS] Question text here [SEP] Ans choice D [SEP]
[CLS] Question text here [SEP] Ans choice E [SEP]
```

The shape of the input will be 5 rows of tokenized values. The label will have a numerical value to indicate which of the answer choices was correct.

In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
lab_order = {"A": 0, "B":1, "C":2, "D":3, "E":4}

class InputExample(object):
    """A single multiple choice question and its five multiple choice answer candidates"""
    # This class is adapted from https://github.com/jonathanherzig/commonsenseqa/blob/master/bert/run_commonsense_qa.py
    # and from https://github.com/rodgzilla/pytorch-pretrained-BERT/blob/dcb50eaa4b80d3ab75d373c36780c80fb47cfd97/examples/run_swag.py

    def __init__(
            self,
            qid,
            question,
            choice_0,
            choice_1,
            choice_2,
            choice_3,
            choice_4,
            label=None):
        """Construct an instance."""
        self.qid = qid
        self.question = question  # e.g., 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?'
        self.choices = [          # All five anser choices as a list
            choice_0,
            choice_1,
            choice_2,
            choice_3,
            choice_4
        ]
        self.label = label        # 
        
    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        l = [
            f"qid: {self.qid}",
            f"question: {self.question}",
            f"choice_0: {self.choices[0]}",
            f"choice_1: {self.choices[1]}",
            f"choice_2: {self.choices[2]}",
            f"choice_3: {self.choices[3]}",
            f"choice_4: {self.choices[4]}",
        ]

        if self.label is not None:
            l.append(f"label: {self.label}")

        return ", ".join(l)    

class InputFeatures(object):
    """Adapted from: https://github.com/rodgzilla/pytorch-pretrained-BERT/blob/dcb50eaa4b80d3ab75d373c36780c80fb47cfd97/examples/run_swag.py
    Stores Bert model inputs (ids, masks) for each example"""
    
    def __init__(self,
                 example_id,
                 choices_features,
                 label

    ):
        self.example_id = example_id
        self.choices_features = [
            {
                'input_ids': input_ids,
                'input_mask': input_mask,
                'segment_ids': segment_ids
            }
            for _, input_ids, input_mask, segment_ids in choices_features
        ]
        self.label = label

    
def process_examples(data):
    """Given the examples in a pandas df format, process examples into example class"""
    examples = []
    labels = []
    questions = []
    anscands = []
    
    
    for index, row in data.iterrows(): 
        example = InputExample(
                    qid=row.id,
                    question=row.stem,
                    choice_0=str(row.choices[0]).replace("'",""),
                    choice_1=str(row.choices[1]).replace("'",""),
                    choice_2=str(row.choices[2]).replace("'",""),
                    choice_3=str(row.choices[3]).replace("'",""),
                    choice_4=str(row.choices[4]).replace("'",""),
                    label=lab_order[row.answerKey]
                )
        examples.append(example)
        
    return examples 

def convert_examples_to_features(examples, tokenizer, max_seq_length, is_training):
    # For each quesiton, we generate five inputs: one for each answer choice. 
    
    # - [CLS] question [SEP] choice_1 [SEP]
    # - [CLS] question [SEP] choice_2 [SEP]
    # - [CLS] question [SEP] choice_3 [SEP]
    # - [CLS] question [SEP] choice_4 [SEP]
    # - [CLS] question [SEP] choice_5 [SEP]
    
    features = []
    # Loop through questions
    for example_index, example in enumerate(examples):
        question_tokens = tokenizer.tokenize(example.question)

        choices_features = []
        # For each question, loop through all answer choices 
        for choice_index, choice in enumerate(example.choices):
            # We create a copy of the question tokens in order to be
            # able to shrink it according to choice_tokens
            question_tokens_choice = question_tokens[:]
            choice_tokens = tokenizer.tokenize(choice)
            # Modifies `question_tokens_choice` and `choice_tokens` in
            # place so that the total length is less than the
            # specified length.  Account for [CLS], [SEP], [SEP] with
            # "- 3"
            _truncate_seq_pair(question_tokens_choice, choice_tokens, max_seq_length - 3)

            tokens = ["[CLS]"] + question_tokens_choice + ["[SEP]"] + choice_tokens + ["[SEP]"]
            segment_ids = [0] * (len(question_tokens_choice) + 2) + [1] * (len(choice_tokens) + 1)

            input_ids = tokenizer.convert_tokens_to_ids(tokens)
            input_mask = [1] * len(input_ids)

            # Zero-pad up to the sequence length.
            padding = [0] * (max_seq_length - len(input_ids))
            input_ids += padding
            input_mask += padding
            segment_ids += padding

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            choices_features.append((tokens, input_ids, input_mask, segment_ids))

        label = example.label
        if example_index < 5:
            logger.info("*** Example ***")
            logger.info(f"qid: {example.qid}")
            for choice_idx, (tokens, input_ids, input_mask, segment_ids) in enumerate(choices_features):
                logger.info(f"choice: {choice_idx}")
                logger.info(f"tokens: {' '.join(tokens)}")
                logger.info(f"input_ids: {' '.join(map(str, input_ids))}")
                logger.info(f"input_mask: {' '.join(map(str, input_mask))}")
                logger.info(f"segment_ids: {' '.join(map(str, segment_ids))}")
            if is_training:
                logger.info(f"label: {label}")

        features.append(
            InputFeatures(
                example_id = example.qid,
                choices_features = choices_features,
                label = label
            )
        )

    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()
            
def select_field(features, field):
    """Yields a list, length equal to the total number of examples,
    where each item is a list of arrays,
    each array representing the feature array"""
    return [
        [
            choice[field]   # Grab the feature array of that choice.
            for choice in feature.choices_features  # Loop through 5 choices of that example
        ]
        for feature in features   # loop through each example
    ]



In [7]:
# Process inputs 
max_seq_length=50
tiny_train = train.iloc[0:10]

train_examples= process_examples(tiny_train)
train_features = convert_examples_to_features(
                    examples=train_examples, 
                    tokenizer=tokenizer, 
                    max_seq_length=max_seq_length, 
                    is_training=True)

all_input_ids = torch.tensor(select_field(train_features, 'input_ids'), dtype=torch.long)
all_input_mask = torch.tensor(select_field(train_features, 'input_mask'), dtype=torch.long)
all_segment_ids = torch.tensor(select_field(train_features, 'segment_ids'), dtype=torch.long)
all_label = torch.tensor([f.label for f in train_features], dtype=torch.long)

train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label)

In [8]:
# You can see that there are as many items as number of examples 
len(train_data)

10

In [9]:
# Each training example has four things
# input id, mask, seg id, and label
len(train_data[0]) 

4

In [10]:
# Input ID has a shape of 5 (num. ans choices) by max length 
train_data[0][0].shape

torch.Size([5, 50])

In [11]:
bert_model='bert-base-uncased'

model = BertForMultipleChoice.from_pretrained(bert_model,
        num_choices = 5
    )

In [12]:
# No idea what these do right now
param_optimizer = list(model.named_parameters())

no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
    ]

In [13]:
num_train_steps = 10
t_total = num_train_steps

In [14]:
learning_rate = 5e-5
warmup_proportion = 0.1

optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=learning_rate,
                         warmup=warmup_proportion,
                         t_total=t_total)

In [15]:
model.train()

BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
           

In [17]:
break here 

SyntaxError: invalid syntax (<ipython-input-17-2ab407b9ec44>, line 1)

In [ ]:
num_train_epochs = 3
train_batch_size =10
train_sampler = RandomSampler(train_data)

train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

for _ in trange(int(num_train_epochs), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    
    input_ids, input_mask, segment_ids, label_ids = all_input_ids,  all_input_mask,  all_segment_ids,  all_label
    
    loss = model(input_ids, segment_ids, input_mask, label_ids)
    
    loss.backward()
    tr_loss += loss.item()
    
    nb_tr_examples += input_ids.size(0)
    nb_tr_steps += 1
    if (step + 1) % args.gradient_accumulation_steps == 0:
        if args.fp16 or args.optimize_on_cpu:
            if args.fp16 and args.loss_scale != 1.0:
                # scale down gradients for fp16 training
                for param in model.parameters():
                    if param.grad is not None:
                        param.grad.data = param.grad.data / args.loss_scale
            is_nan = set_optimizer_params_grad(param_optimizer, model.named_parameters(), test_nan=True)
            if is_nan:
                logger.info("FP16 TRAINING: Nan in gradients, reducing loss scaling")
                args.loss_scale = args.loss_scale / 2
                model.zero_grad()
                continue
            optimizer.step()
            copy_optimizer_params_to_model(model.named_parameters(), param_optimizer)
        else:
            optimizer.step()
        model.zero_grad()
        global_step += 1

Process dev and training data. We'll just do the first few to make a tiny dataset.

In [ ]:
dev_eg, dev_encoded_eg, dev_labs = process_examples(dev[0:2])
train_eg, train_encoded_eg, train_labs = process_examples(train.iloc[0:2])

### Game plan

I'll be following the tutorial on: http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/, but for specifically commonsenseQA. Here are the steps from the tutorial.

1. Embed all sentences. Let's look at the output from the BERT model for our inputs. 
2. The tutorial says do a train/test split. We don't need this 'cause our data came separate.
3. Train the logistic regressio model using the training set. This is training on the output of BERT. I will need to create a FFNN to attach at the end of BERT. See what it comes back with. 
4. (Optional for now) For each question, pick the answer with the highest score. 
5. Then, evaluate against true answers. The evaluation metric will be % of questions with the correct answers out of all questions. 


#### Step 1: Embed all sentences. 

I've already formatted the CommonsenseQA inpts to be fed into the BERT model. Let's look at the output from the BERT model for our inputs. It should have a 768-long vector for each input token.

### Sunday morning retry 

In [ ]:
NAME = 'oob_bert_classify_{runtype}_{ts}'.format(runtype=runtype, ts=int(time.time()))
tensorboard = TensorBoard(log_dir='models/logs/{}'.format(NAME))

def classification_model(max_len):
    """Implementation of classification model.
    Returns: model"""
    ## BERT encoder
    encoder = TFBertModel.from_pretrained("bert-base-uncased")
    ## QA Model
    input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="InputID")
    attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="AttentionMask")
    token_type_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="TokenTypeID")
    embedding = encoder(
        [input_ids, attention_mask, token_type_ids]
    )[0]
    # Feed inputs through the bert model, 
    # then take just the vector associated with first token [CLS]
    bert_cls_output = embedding[:,0]
    # These are the layers that come after Bert.
    dense = tf.keras.layers.Dense(256, activation='relu', name='dense')(bert_cls_output)
    # Output layer to predict correct answer. 
    # For the future, we may modify it to choose the max candidate answer of each question
    # for now, just predict from 0 to 1 whether this looks like a correct answer. 
    pred = tf.keras.layers.Dense(1, activation='sigmoid', name='correct')(dense)
    model = tf.keras.models.Model(inputs=[input_ids, token_type_ids, attention_mask],
                                  outputs=pred)
    model.compile(loss="binary_crossentropy", 
                  optimizer="adam",
                  metrics=["accuracy"])
    model.summary()
    return model


# Train

On training set

In [ ]:
# train_eg, train_encoded_eg, train_labs 

max_length= len(train_encoded_eg[0])   # num max token 
BertClassifierModel = classification_model(max_len=max_length)


In [ ]:
start = time.time()

BertClassifierModel.fit(
    [   train_encoded_eg[0],
        train_encoded_eg[1],
        train_encoded_eg[2]],
    train_labs, 
    epochs=3,
    # Insert validation data
    validation_data=(
        [dev_encoded_eg[0],
         dev_encoded_eg[1],
         dev_encoded_eg[2]
        ], dev_labs
    ),
    # Log the training info on tensorboard
    callbacks=[tensorboard])

end = time.time()
print("Execution duration in minutes:", (end - start)/60)

In [ ]:
predictions = BertClassifierModel.predict(
    [dev_encoded_eg[0],
         dev_encoded_eg[1],
         dev_encoded_eg[2]
        ])

location="models/predictions/BertClassifierModel_{}_dev_predictions".format(runtype)
pickle_out = open(location, "wb")
pickle.dump(predictions, pickle_out)
pickle_out.close()

In [ ]:
location='models/{NAME}.h5'.format(NAME=NAME)

BertClassifierModel.save(location)